In [2]:
import json
import re
from collections import Counter
from collections import defaultdict

In [3]:
with open('/home/thoang/workspace/spert/data/datasets/scierc/scierc_train.json', 'r') as f:
    scierc = json.load(f)

In [10]:
with open('/home/thoang/workspace/spert/data/core_scierc/scierc_10/fold_1/unlabeled_w_labels.json', 'r') as f:
    uscierc = json.load(f)

In [4]:
len(scierc)

1861

In [13]:
uscierc[1035]

{'tokens': ['This',
  'paper',
  'describes',
  'an',
  'experiment',
  'which',
  'confirmed',
  'this',
  'hypothesis',
  'and',
  'found',
  'that',
  'the',
  'tendency',
  'to',
  'share',
  'sense',
  'in',
  'the',
  'same',
  'discourse',
  'is',
  'extremely',
  'strong',
  '(',
  '98',
  '%',
  ')',
  '.'],
 'entities': [{'type': 'OtherScientificTerm', 'start': 20, 'end': 21}],
 'relations': [],
 'orig_id': '22c4ce07-d882-47e4-8e29-7ed0d23e8d92'}

In [5]:
count = 0
for i, row in enumerate(scierc):
    rel_types = [r['type'] for r in row['relations']]
    if 'Conjunction' in rel_types:
        count += 1
        print('-------------------')
        print(i)
        tokens = row['tokens']
        print(tokens)
        entities = row['entities']
        for e in row['entities']:
            start, end, etype = e['start'], e['end'], e['type']
            w = ' '.join(tokens[start: end])
            print(f"{etype}({w})-({start},{end})")
        print('+++++')
        
        for r in row['relations']:
            head, tail, rtype = entities[r['head']], entities[r['tail']], r['type']
            hw = ' '.join(tokens[head['start']: head['end']])
            tw = ' '.join(tokens[tail['start']: tail['end']])
            print(f"{rtype}({hw}, {tw}) - ({head['start']},{head['end']},{tail['start']},{tail['end']})")

-------------------
1
['The', 'agreement', 'in', 'question', 'involves', 'number', 'in', 'nouns', 'and', 'reflexive', 'pronouns', 'and', 'is', 'syntactic', 'rather', 'than', 'semantic', 'in', 'nature', 'because', 'grammatical', 'number', 'in', 'English', ',', 'like', 'grammatical', 'gender', 'in', 'languages', 'such', 'as', 'French', ',', 'is', 'partly', 'arbitrary', '.']
Generic(agreement)-(1,2)
OtherScientificTerm(nouns)-(7,8)
OtherScientificTerm(reflexive pronouns)-(9,11)
OtherScientificTerm(grammatical number)-(20,22)
Material(English)-(23,24)
OtherScientificTerm(grammatical gender)-(26,28)
Material(languages)-(29,30)
Material(French)-(32,33)
+++++
Conjunction(nouns, reflexive pronouns) - (7,8,9,11)
Feature-of(grammatical gender, languages) - (26,28,29,30)
Hyponym-of(French, languages) - (32,33,29,30)
-------------------
9
['Experiments', 'show', 'that', 'the', 'learned', 'tracker', 'performs', 'much', 'better', 'than', 'existing', 'trackers', 'on', 'the', 'tracking', 'of', 'comple

In [6]:
count

296

In [5]:
match_head = []
match_tail = []
match_else = []
for row in scierc:
    for i, ent in enumerate(row['entities']):
        for j, a_ent in enumerate(row['entities']):
            if i != j:
                if a_ent['start'] >= ent['start'] and a_ent['end'] <= ent['end']:
                    flag = False
                    print(f"({a_ent['start']},{a_ent['end']}) inside ({ent['start']},{ent['end']})")
                    
                    if a_ent['start'] == ent['start']:
                        print('head')
                        match_head.append((ent['type'], a_ent['type']))
                    elif a_ent['end'] == ent['end']:
                        print('tail')
                        match_tail.append((ent['type'], a_ent['type']))
                    else:
                        print('else')
                        match_else.append((ent['type'], a_ent['type']))
                        print(row['tokens'][a_ent['start']:a_ent['end']])
                        print(row['tokens'][ent['start']:ent['end']])
                    print('--------------')

(19,22) inside (16,22)
tail
--------------
(16,19) inside (14,19)
tail
--------------
(6,7) inside (6,10)
head
--------------
(17,19) inside (15,19)
tail
--------------
(10,12) inside (8,12)
tail
--------------
(22,25) inside (19,25)
tail
--------------
(6,9) inside (6,13)
head
--------------
(9,10) inside (6,10)
tail
--------------
(28,30) inside (25,30)
tail
--------------
(19,21) inside (16,21)
tail
--------------
(14,15) inside (10,15)
tail
--------------
(6,8) inside (3,8)
tail
--------------
(15,16) inside (13,16)
tail
--------------
(6,7) inside (4,7)
tail
--------------
(7,8) inside (4,8)
tail
--------------
(9,13) inside (8,13)
tail
--------------
(34,36) inside (32,36)
tail
--------------
(3,5) inside (0,5)
tail
--------------
(14,16) inside (13,16)
tail
--------------
(31,33) inside (29,33)
tail
--------------
(32,37) inside (28,37)
tail
--------------
(2,3) inside (2,7)
head
--------------
(4,5) inside (2,7)
else
['IA']
['QCNs', 'of', 'IA', 'and', 'RCC-8']
--------------
(6

In [6]:
print(len(match_head))
print(len(match_tail))
print(len(match_else))

8
50
3
